# Change default settings

This Notebook outlines the settings for the potentials package that can be saved across sessions.  Doing so provides users a convenient means of saving useful parameter sets and to control default behaviors.   

1. __Settings basics__ describes how the settings are saved and where the settings file is located. 
2. __Default options__ describes which Database class parameters default values can be set for.
3. __Database interactions__ describes how to change the databases that the Database class interacts with.

All of the settings here are controlled by using potentials.settings.

In [1]:
import potentials

## 1. Settings basics 

The potentials package handles user-defined settings by creating a settings.json file. The default location for this file is in a directory called ".NISTpotentials" located in the user's home directory.

This location is usually suitable for most users, but can be changed using __set_directory()__.  One example of when changing the directory could be useful is if multiple environments that each define different home directories are being used, and you wish all of the environments to access the same settings. 

__NOTE__: the directory can only be changed if no other settings have been modified!

The settings directory can be reverted back to the default by using __unset_directory()__.

In [3]:
print(potentials.settings.directory)
print(potentials.settings.filename)

C:\Users\lmh1\.NISTpotentials
C:\Users\lmh1\.NISTpotentials\settings.json


In [ ]:
potentials.settings.set_directory('C:/Users/lmh1/test')
print(potential.settings.filename)

In [ ]:
potential.settings.unset_directory()
print(potential.settings.directory)

## 2. Default options

The default values of some optional parameters of the potentials.Database class can also be changed.  Setting these may be useful depending on how you primarily intend to use the potentials package.  Any of the default values set here can be overridden when using the potentials.Database class, so changing these settings is entirely optional. 

### 2.1. Accessing database

The potentials.Database class is designed to allow for simultaneous interactions with two distinct database locations: the remote database hosted at https://potentials.nist.gov, and a local personal database.  The "remote" and "local" parameters accept boolean values allowing for interactions with either of these locations to turned on/off. The default values for these parameters can be checked with __remote__ and __local__, and changed using __set_remote()__ and __set_local()__.

__NOTE__: It is recommended for most users that these default settings be left as True and to explicitly turn off an interaction when initializing a Database or querying records.  Changing the default values here should ideally only be done if potentials is being integrated into a project where remote-only or local-only interactions are done.

In [8]:
# Set default local and remote values
potentials.settings.set_local(True)
potentials.settings.set_remote(True)

# Check their values
print('local = ', potentials.settings.local)
print('remote =', potentials.settings.remote)

local =  True
remote = True


### 2.2. LAMMPS potential directory styles

The majority of native LAMMPS potentials hosted by the NIST Interatomic Potentials Repository are defined using parameter files.  The __pot_dir_style__ parameter sets the default location(s) where the paramter files are expected to be when LAMMPS simulations are performed, and where the associated parameter files are downloaded/copied to when needed.  The default value for this option is controlled by the settings method __set_pot_dir_style()__.

Three styles are allowed for this setting:

- __"working"__ uses the working directory.  This is the simplest option for simulations where only one potential is to be used.  Any retrieved parameter files are copied to the working directory making it convenient to archive the parameter file with the other inputs and results for the simulations. 
- __"id"__ uses a subdirectory of the working directory matching the LAMMPS potential's id.  This also allows for the parameter files to be archived with the simulation inputs and results, but also avoids any issues if parameter files of the same name are used by different potentials.
- __"local"__ uses directories within the local database location.  The advantage of this option is that only one copy of each unique parameter file is stored on your computer, and no copies/downloads need to be performed when running new simulations.  Note that this style only works if the local database is of "local" style.

In [9]:
#potentials.settings.set_pot_dir_style('working')
potentials.settings.set_pot_dir_style('id')
#potentials.settings.set_pot_dir_style('local')

print('pot_dir_style =', potentials.settings.pot_dir_style)

pot_dir_style = id


### 2.3. KIM models

In order for models from openKIM to be included alongside native LAMMPS potentials, the potentials package needs to know which kim models to include.  The list can be defined during Database initialization, or default options can be saved using settings for convenience.  The two related settings options are

- __set_kim_models()__ allows for a list of kim models to be explicitly given and saved to a file __kim_models_file__ in the settings directory. 
- __set_kim_api_directory()__ allows for a default __kim_api_directory__ value to be set. This allows for potentials to use the kim-api-collections-management command in kim_api_directory to automatically determine the list of currently installed kim models.

__NOTE__: If both default options are set, the explicit list of kim models in kim_models_file will be used over the kim api. Either setting can be cleared using the corresponding unset method: __unset_kim_models()__ or __unset_kim_api_directory()__.

In [10]:
print('kim_models_file =  ', potentials.settings.kim_models_file)
print('kim_api_directory =', potentials.settings.kim_api_directory)

kim_models_file =   C:\Users\lmh1\.NISTpotentials\kim_models.txt
kim_api_directory = None


In [ ]:
kim_models = []
potentials.settings.set_kim_models(kim_models)
potentials.settings.unset_kim_api_directory()

In [ ]:
dirpath = ''
potentials.settings.set_kim_api_directory(dirpath)
potentials.settings.unset_kim_models(kim_models)

## 3. Database interactions

The specific remote and local databases that are accessed by the potentials.Database class can also be changed.  The advantages of this are that

- A different local directory than the one in the user's settings directory can be used.
- A different remote database than potentials.nist.gov can be used, if an alternate site ever exists.
- The style of the local database can be changed, for instance a personal MongoDB can be used instead.

The first point is the only one that most users would be interested in. As such, there is a simple settings option for those users, while full database control options are open for more complicated changes.

### 3.1. Simple local database settings

The __set_local_database()__ method allows for the local database directory and file format options to be changed.  The parameters that the set method takes are 

- __localpath__ (*path, optional*) The default directory to use for the local database.  If not given, will use the default path of directory "library" located in the settings directory.
- __format__ (*str, optional*) The file format that is used: "json" or "xml".  Default value is "json"
- __indent__ (*int or None, optional*) The number of indentation spacings to use in the files.  If None, then the files will be compact.  Default value is 4.

In [ ]:
potentials.settings.set_local_database(localpath='', format='json', indent=4)

### 3.2. Full database setting options

The potentials.Database class actually interacts with two distinct databases: the local and the remote. Interaction settings for these two databases, as well as others, can be defined and saved to the settings file. 

The related settings attributes and parameters are

- __set_database()__ allows for database interaction settings to be saved under a given name.
- __unset_database()__ allows for the saved database interaction settings for a given name to be deleted.
- __list_databases__ returns a list of the database interaction names.
- __databases__ returns a dictionary with keys matching the database interaction names and values being a dictionary of all interaction parameter settings for that database.

Then, the interactions used by the potentials.Database are specified according to:

- Default local and remote database settings will use interaction settings saved under the names "potentials_local" and "potentials_remote", respectively, if given.  Otherwise will use the default default values of remote being an anonymous visitor to potentials.nist.gov, and local being the "library" directory in the settings directory.
- Alternate saved settings can be loaded and used by giving the corresponding database interaction names to the local_name and remote_name parameters.
- Or, all database interaction settings given manually every time a new potentials.Database is created.


In [11]:
potentials.settings.list_databases

['local']

__set_database()__ takes the parameters

- __name__
- __style__
- __host__ 
- __\*\*kwargs__